## Preprocessing

In [1]:
!gdown 1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV
!unzip -q icdar2003.zip -d datasets

Downloading...
From (original): https://drive.google.com/uc?id=1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV
From (redirected): https://drive.google.com/uc?id=1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV&confirm=t&uuid=7fe22c05-0723-491c-a2f1-d5ed0afd967d
To: /content/icdar2003.zip
100% 45.7M/45.7M [00:00<00:00, 88.4MB/s]


In [2]:
%pip install -q -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.3/904.3 kB 50.5 MB/s eta 0:00:00


In [3]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/112.6 GB disk)


In [25]:
import os
import shutil
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import yaml

In [5]:
def extract_data(path):
    tree = ET.parse(path)
    root = tree.getroot()

    im_paths, im_sizes, im_labels, im_bboxes = [], [], [], []
    for im in root:
        bboxes = []
        labels = []

        for bbs in im.findall('taggedRectangles'):
            for bb in bbs:
                if not bb[0].text.isalnum():
                    continue
                if "é" in bb[0].text.lower() or "ñ" in bb[0].text.lower():
                    continue

                bboxes.append(
                    [float(bb.attrib['x']),
                     float(bb.attrib['y']),
                     float(bb.attrib['width']),
                     float(bb.attrib['height'])]
                )
                labels.append(bb[0].text.lower())

        im_paths.append(im[0].text)
        im_sizes.append((int(im[1].attrib['x']), int(im[1].attrib['y'])))
        im_labels.append(labels)
        im_bboxes.append(bboxes)

    return im_paths, im_sizes, im_labels, im_bboxes

In [6]:
def convert_to_yolo_format(im_paths, im_sizes, im_bboxes):
    yolo_data = []
    for im_path, im_size, im_bbox in zip(im_paths, im_sizes, im_bboxes):
        im_width, im_height = im_size
        yolo_labels = []

        for bbox in im_bbox:
            x, y, w, h = bbox # (topleft_x, topleft_y, width, height)
            x_center = (x + w / 2) / im_width
            y_center = (y + h / 2) / im_height
            w /= im_width
            h /= im_height
            class_id = 0

            # Convert to YOLO format
            yolo_labels.append(f'{class_id} {x_center} {y_center} {w} {h}')
        yolo_data.append((im_path, yolo_labels))
    return yolo_data

In [ ]:
# Save data to YOLO format
def save_data(data, src_im_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(os.path.join(save_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, 'labels'), exist_ok=True)

    for im_path, labels in data:
        shutil.copy(os.path.join(src_im_dir, im_path), os.path.join(save_dir, 'images'))
        im_name = os.path.basename(im_path).split('.')[0]
        with open(os.path.join(save_dir, 'labels', f'{im_name}.txt'), 'w') as f:
            for label in labels:
                f.write(f'{label}\n')


In [20]:
path = './datasets/SceneTrialTrain/words.xml'
class_labels = ['text']
im_paths, im_sizes, im_labels, im_bboxes = extract_data(path)
yolo_data = convert_to_yolo_format(im_paths, im_sizes, im_bboxes)

In [ ]:
seed = 0

# 7:2:1
val_size = 0.2
test_size = 0.125
train_data, val_data = train_test_split(yolo_data, test_size=val_size, random_state=seed, shuffle=True)
train_data, test_data = train_test_split(train_data, test_size=test_size, random_state=seed, shuffle=True)

In [22]:
print(len(train_data), len(val_data), len(test_data))

175 50 25


In [ ]:
data_dir = './datasets/SceneTrialTrain'
save_dir = './datasets/yolo_data'
save_train_dir = os.path.join(save_dir, 'train')
save_val_dir = os.path.join(save_dir, 'val')
save_test_dir = os.path.join(save_dir, 'test')

save_data(train_data, data_dir, save_train_dir)
save_data(val_data, data_dir, save_val_dir)
save_data(test_data, data_dir, save_test_dir)

In [45]:
print(len(os.listdir(os.path.join(save_train_dir, 'images'))))
print(len(os.listdir(os.path.join(save_train_dir, 'labels'))))
print(len(os.listdir(os.path.join(save_val_dir, 'images'))))
print(len(os.listdir(os.path.join(save_val_dir, 'labels'))))

175
175
50
50


In [ ]:
# Create yml file for YOLO training
data_yml = {
    'path': '/content/datasets/yolo_data',
    'train': '/content/datasets/yolo_data/train/images',
    'val': '/content/datasets/yolo_data/val/images',
    'test': '/content/datasets/yolo_data/test/images',
    'nc': 1,
    'names': class_labels
}
yolo_yml_path = os.path.join(save_dir, 'data.yml')
with open(yolo_yml_path, 'w') as f:
    yaml.dump(data_yml, f, default_flow_style=False)

## Training

In [55]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')

results = model.train(
    data=yolo_yml_path,
    epochs=100,
    imgsz=640,
    cache=True,
    patience=20,
    plots=True,
)

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=./datasets/yolo_data/data.yml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

100%|██████████| 5.35M/5.35M [00:00<00:00, 331MB/s]


AMP: checks passed ✅


train: Scanning /content/datasets/yolo_data/train/labels... 173 images, 7 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<00:00, 1203.55it/s]

train: New cache created: /content/datasets/yolo_data/train/labels.cache


...

     69/100      8.68G     0.7335     0.5219     0.9711        121        640: 100%|██████████| 11/11 [00:06<00:00,  1.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.45it/s]

                   all         50        262      0.893      0.805       0.88      0.643
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



69 epochs completed in 0.173 hours.
Optimizer stripped from runs/detect/train13/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train13/weights/best.pt, 40.5MB

Validating runs/detect/train13/weights/best.pt...
Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.45it/s]


                   all         50        262      0.831      0.855      0.877      0.648
Speed: 0.2ms preprocess, 9.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train13


In [56]:
model_path = 'runs/detect/train13/weights/best.pt'
model = YOLO(model_path)
metrics = model.val()

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning /content/datasets/yolo_data/val/labels.cache... 50 images, 2 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.70it/s]


                   all         50        262      0.833      0.855      0.879      0.647
Speed: 0.4ms preprocess, 27.0ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/detect/val
